## PINN of 2D Navier-Stokes equation

In this project we consider the following 2D Navier-Stokes equation,
\begin{equation}
\begin{split}
&u_t + \lambda_1(u u_x + v u_y) = -p_x + \lambda_2 (u_{xx} + u_{yy})
\\
&v_t + \lambda_1(u v_x + v v_y) = -p_y + \lambda_2 (v_{xx} + v_{yy})
\\
&u_x+v_y=0
\end{split}
\end{equation}

Here $u$, $v$ are two components of the velocity field, $p$ is the pressure field. The ground truth values of $\lambda_1$ and $\lambda_2$ are $1$ and $0.01$ respectively. The ground truth values of $u$, $v$, $p$ are stored in 'cylinder_nektar_wake.mat'.

We solve this equation by PINN method. In this method, a neural network is introduced to approximate the solution and parameters in the network is find by the gradient descent method.

To simplify the equation, we introduce the so called stream function $\psi$. Actually, the third equation in above can be solved. All solutions $u$, $v$ of it are of the form:
\begin{equation}
u= -\psi_y,\qquad v=\psi_x
\end{equation}

Therefore the Navier-Stokes equation is an equation of $\psi$ and $p$.

Finally we solve the following equation of $\psi$ and $p$
\begin{equation}
\begin{split}
&u_t + \lambda_1(u u_x + v u_y) = -p_x + \lambda_2 (u_{xx} + u_{yy})
\\
&v_t + \lambda_1(u v_x + v v_y) = -p_y + \lambda_2 (v_{xx} + v_{yy})
\\
&u= -\psi_y,\qquad v=\psi_x
\end{split}
\end{equation}
Substituting the last two equations into the first two removes $u$ and $v$ completely.

Now let's go into the detail of the code, let's first import some packages.

In [43]:
import torch
import numpy as np
# import matplotlib.pyplot as plt
import scipy.io
# from scipy.interpolate import griddata
import time
# from itertools import product, combinations
# from mpl_toolkits.mplot3d import Axes3D
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection
# # from plotting import newfig, savefig
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# import matplotlib.gridspec as gridspec
# from matplotlib import rc

np.random.seed(123)
torch.manual_seed(100)

print(torch.__version__)

1.11.0+cu102


The definition of the neural network are provided below. Before go into the code, we provide some remarks.

1. **Organization of the data, the `__init__` function.** The data `xyt`, `u`, `v` are stored in 'cylinder_nektar_wake.mat' and their code can be find in the next code block. `xyt` is the spacetime location of the sample points and `u`, `v` are values of velocity fields components at these sample points. When sampling the data points, we first choose `T` time slices $\{t_j\}_{j=1,\cdots,T}$ and on each time slice we choose `N` points $\{(x_i, y_i)\}_{i=1,\cdots,N}$, finally on each data point $(x_i, y_i,t_j)$ we are given $u_{ij}=u(x_i, y_i,t_j)$ and $v_{ij}=v(x_i, y_i,t_j)$. 

2. **`initialize_NN` and `xavier_init` function.** We randomly initialize the weights the neural networks, using the Xavier initialization method, which sets each entry of the weights matrices `weights[i]` to be a Gaussian variable of mean $0$ and standard deviation $\sqrt{\frac{2}{in\_ dim+out\_ dim}}$ (assume that `weights[i].size()=[in_dim, out_dim]`).


3. **`neural_net` and `net_NS` function.** `neural_net` defines a fully connected neural network and the size of each layer is given by `layers=[3,layers[1],...,layers[i-1], 2]`. When calling `neural_net` in `net_NS`, the input of this network is a `[N*T,3]` tensor `xyt` and the output is a `[N*T,2]` tensor `psi_and_p`. The first components of the output `psi_and_p[:,0]` is $\psi$ and The first components `psi_and_p[:,1]` is $p$. The output of `net_NS` is `u`, `v`, `p`, `f_u`, `f_v` which is used to calculate the loss function. Remember that $u$ and $v$ can be calculated from $\psi$ by $u= -\psi_y$, $v=\psi_x$. We take this derivative using numerical derivative $f'(x)=\frac{f(x+\Delta)-f(x)}{\Delta}$. In the code `u = (self.neural_net(xyt+Delta*e_y, self.weights, self.biases) - self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta`, `self.neural_net(xyt+Delta*e_y, self.weights, self.biases)[:,0]` calculates $\psi(x+\Delta)$.

4. **`loss_function` function.** The first two terms are data losses and the second two terms are equation losses.

In [49]:
# Model
import torch.nn as nn
from torch.autograd import grad
from torch.autograd import backward

class PhysicsInformedNN(nn.Module):
    def __init__(self, xyt, u, v, layers): # xyt.size()=(N*T,3), Xbatch=N*T
        super().__init__()
        self.xyt = xyt
        self.u = u
        self.v = v
        
        self.lb = xyt.min()
        self.ub = xyt.max()

        self.layers = layers

        self.weights, self.biases = self.initialize_NN(layers)  
        
        # Initialize parameters
        self.lambda_1 = nn.Parameter(torch.zeros(1, dtype=torch.float64))
        self.lambda_2 = nn.Parameter(torch.zeros(1, dtype=torch.float64))

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = nn.Parameter(torch.zeros([layers[l+1]], dtype=torch.float64))
            weights.append(W)
            biases.append(b) 
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return nn.Parameter(xavier_stddev*torch.randn([in_dim, out_dim], dtype=torch.float64))
    
    def neural_net(self, X, weights, biases): # X.size()=(Xbatch,3), 
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0     
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = torch.tanh(H@W+b)
        W = weights[-1]
        b = biases[-1]
        Y = H@W+b
        return Y
    
    def net_NS(self, xyt): # xyt.size()=(N*T,3) Xbatch=N*T
        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2
        
        psi_and_p = self.neural_net(xyt, self.weights, self.biases)
        psi = psi_and_p[:,0] # psi.size(), p.size()=N*T
        p = psi_and_p[:,1]

        Delta = 0.001
        
        # calculate u,v, p_x, p_y
        e_x = torch.tensor([[1,0,0]]).tile((xyt.size(0),1))
        e_y = torch.tensor([[0,1,0]]).tile((xyt.size(0),1))
        e_t = torch.tensor([[0,0,1]]).tile((xyt.size(0),1))
        u = (self.neural_net(xyt+Delta*e_y, self.weights, self.biases) -\
            self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta # u = psi_y
        v = -(self.neural_net(xyt+Delta*e_x, self.weights, self.biases) -\
            self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta # v = -psi_x
        
        p_x = (self.neural_net(xyt+Delta*e_x, self.weights, self.biases) -\
            self.neural_net(xyt, self.weights, self.biases) )[:,1]/Delta
        p_y = (self.neural_net(xyt+Delta*e_y, self.weights, self.biases) -\
            self.neural_net(xyt, self.weights, self.biases) )[:,1]/Delta
        p_t = (self.neural_net(xyt+Delta*e_t, self.weights, self.biases) -\
            self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta
        
        
        # calculate u_x, u_y, u_t; v_x, v_y, v_t
        
        u_x = (self.neural_net(xyt+Delta*e_x+Delta*e_y, self.weights, self.biases) - \
            self.neural_net(xyt+Delta*e_x, self.weights, self.biases)-\
            self.neural_net(xyt+Delta*e_y, self.weights, self.biases)+\
            self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # u_x = psi_xy
        u_y = (self.neural_net(xyt+Delta*e_y, self.weights, self.biases) + \
            self.neural_net(xyt-Delta*e_y, self.weights, self.biases)-\
            2*self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # u_y = psi_yy
        u_t = (self.neural_net(xyt+Delta*e_y+Delta*e_t, self.weights, self.biases) - \
            self.neural_net(xyt+Delta*e_y, self.weights, self.biases)-\
            self.neural_net(xyt+Delta*e_t, self.weights, self.biases)+\
            self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # u_t = psi_ty


        
        v_x = - (self.neural_net(xyt+Delta*e_x, self.weights, self.biases) + \
            self.neural_net(xyt-Delta*e_x, self.weights, self.biases)-\
            2*self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # v_x = - psi_xx
        v_y = - (self.neural_net(xyt+Delta*e_x+Delta*e_y, self.weights, self.biases) - \
            self.neural_net(xyt+Delta*e_x, self.weights, self.biases)-\
            self.neural_net(xyt+Delta*e_y, self.weights, self.biases)+\
            self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # v_x = - psi_xy
        v_t = - (self.neural_net(xyt+Delta*e_x+Delta*e_t, self.weights, self.biases) - \
            self.neural_net(xyt+Delta*e_x, self.weights, self.biases)-\
            self.neural_net(xyt+Delta*e_t, self.weights, self.biases)+\
            self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # v_x = - psi_tx
        
        # calculate u_xx, u_yy; v_xx, v_yy
        
        u_xx = (self.neural_net(xyt+Delta*e_x+Delta*e_y, self.weights, self.biases)+\
            self.neural_net(xyt-Delta*e_x+Delta*e_y, self.weights, self.biases)- \
            2*self.neural_net(xyt+Delta*e_y, self.weights, self.biases)-
            self.neural_net(xyt+Delta*e_x, self.weights, self.biases)-\
            self.neural_net(xyt-Delta*e_x, self.weights, self.biases)+\
            2*self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # u_xx = psi_xxy

        u_yy = (0.5*self.neural_net(xyt+2*Delta*e_y, self.weights, self.biases) - \
            self.neural_net(xyt+Delta*e_x, self.weights, self.biases)+\
                self.neural_net(xyt+Delta*e_y, self.weights, self.biases)-\
            0.5*self.neural_net(xyt-2*Delta*e_y, self.weights, self.biases) )[:,0]/Delta**2 # u_yy = psi_yyy
        
        v_xx = -(0.5*self.neural_net(xyt+2*Delta*e_x, self.weights, self.biases) - \
            self.neural_net(xyt+Delta*e_x, self.weights, self.biases)+\
                self.neural_net(xyt+Delta*e_x, self.weights, self.biases)-\
            0.5*self.neural_net(xyt-2*Delta*e_x, self.weights, self.biases) )[:,0]/Delta**2 # v_xx = - psi_xxx
            
        v_yy = -(self.neural_net(xyt+Delta*e_x+Delta*e_y, self.weights, self.biases)+\
            self.neural_net(xyt+Delta*e_x-Delta*e_y, self.weights, self.biases)- \
            2*self.neural_net(xyt+Delta*e_x, self.weights, self.biases)-
            self.neural_net(xyt+Delta*e_y, self.weights, self.biases)-\
            self.neural_net(xyt-Delta*e_y, self.weights, self.biases)+\
            2*self.neural_net(xyt, self.weights, self.biases) )[:,0]/Delta**2 # v_yy = - psi_xyy
        
        # calculate two components of f
        f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy) 
        f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy)
        
        return u, v, p, f_u, f_v
    
    def forward(self, xyt):
        return self.net_NS(xyt)
    
    def loss_function(self):
        return  (self.u.squeeze() - self.forward(self.xyt)[0]).pow(2).sum()+\
                (self.v.squeeze() - self.forward(self.xyt)[1]).pow(2).sum()+\
                self.forward(self.xyt)[3].pow(2).sum()+\
                self.forward(self.xyt)[4].pow(2).sum()

This is another version of `net_NS` function use higher order derivative in pytorch. As you can see, the result of pytorch derivative `u[65]` and numerical derivative `u_diff[65]` are completely different but `u[65]` and `u_diff[65]` are exactly the same.

In [50]:
import copy

class PhysicsInformedNN_pytorch(PhysicsInformedNN):        
        def net_NS(self, xyt): # xyt.size()=(N*T,3) Xbatch=N*T
                self.create_graph = True
                self.retain_graph = True
                delta = 1e-7

                lambda_1 = self.lambda_1
                lambda_2 = self.lambda_2

                psi_and_p = self.neural_net(xyt, self.weights, self.biases)
                psi = psi_and_p[:,0] # psi.size(), p.size()=N*T
                p = psi_and_p[:,1]

                # calculate u,v, p_x, p_y
                Dpsi = grad(psi, xyt, grad_outputs=torch.ones_like(psi), create_graph=self.create_graph, retain_graph=self.retain_graph)
                Dp = grad(p, xyt, grad_outputs=torch.ones_like(p), create_graph=self.create_graph, retain_graph=self.retain_graph)

                psi_x = Dpsi[0][:,0]
                psi_y = Dpsi[0][:,1]
                psi_t = Dpsi[0][:,2]
                u = psi_y
                e_0 = torch.tensor([[0,1,0]]).tile((xyt.size(0),1))
                u_diff=1./delta*(self.neural_net(xyt+delta*e_0, self.weights, self.biases) -\
                        self.neural_net(xyt, self.weights, self.biases) )[:,0]
#                 print("Total difference between numerical derivative and pytorch derivative: %e" % ((u_diff-u).pow(2).sum().squeeze()))
#                 for idx, (u_d_i, u_i) in enumerate(zip(u_diff, u)):
#                     if torch.abs(u_d_i - u_i)>1e-1:
#                         print(f"u_diff-u: {idx}, {u_d_i-u_i}, {u_d_i:.6f}, {u_i:.6f}")

#                 print("u_diff[65]-u[65]: %e" %(u_diff[65]-u[65]) )
#                 print("u_diff[65]: %e" %(u_diff[65]))
#                 print("u[65]: %e" %(u[65]))
#                 print("u_diff[66]-u[66]: %e" %(u_diff[66]-u[66]) )
#                 print("u_diff[66]: %e" %(u_diff[66]))
#                 print("u[66]: %e" %(u[66]))


                v = -psi_x

                p_x = Dp[0][:,0]
                p_y = Dp[0][:,1]
                p_t = Dp[0][:,2]


                # calculate u_x, u_y, u_t; v_x, v_y, v_t
                Du = grad(u, xyt, grad_outputs=torch.ones_like(u), create_graph=self.create_graph, retain_graph=self.retain_graph)
                Dv = grad(v, xyt, grad_outputs=torch.ones_like(v), create_graph=self.create_graph, retain_graph=self.retain_graph)

                u_x = Du[0][:,0]
                u_y = Du[0][:,1]
                u_t = Du[0][:,2]

                v_x = Dv[0][:,0]
                v_y = Dv[0][:,1]
                v_t = Dv[0][:,2]

                # calculate u_xx, u_yy; v_xx, v_yy
                Du_x = grad(u_x, xyt, grad_outputs=torch.ones_like(u), create_graph=self.create_graph, retain_graph=self.retain_graph)
                Du_y = grad(u_y, xyt, grad_outputs=torch.ones_like(u), create_graph=self.create_graph, retain_graph=self.retain_graph)
                Dv_x = grad(v_x, xyt, grad_outputs=torch.ones_like(u), create_graph=self.create_graph, retain_graph=self.retain_graph)
                Dv_y = grad(v_y, xyt, grad_outputs=torch.ones_like(u), create_graph=self.create_graph, retain_graph=self.retain_graph)

                u_xx = Du_x[0][:,0]
                u_yy = Du_y[0][:,1]

                v_xx = Dv_x[0][:,0]
                v_yy = Dv_y[0][:,1]

                # calculate two components of f
                f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy) 
                f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy)
                
                return u, v, p, f_u, f_v
        
        def forward(self, xyt):
                return self.net_NS(xyt)

In code block below, we prepare the training data `xyt` that will be the input of the network.  

This `xyt` is a tensor of the size (N*T, 3), `xyt[i]` is of the form `tensor([x_i, y_i, t_i])`. In `xyt` x coordinate changes first, y coordinate changes second and z coordinate changes last. One example of `xyt` is `xyt=tensor([[1.0, 1.0, 1.0], [1.5, 1.0, 1.0], [2.0, 1.0, 1.0], [1.0, 1.5, 1.0], [1.5, 1.5, 1.0], [2.0, 1.5, 1.0], [1.0, 2.0, 1.0], [1.5, 2.0, 1.0], [2.0, 2.0, 1.0], [1.0, 1.0, 1.5], [1.5, 1.0, 1.5], [2.0, 1.0, 1.5], [1.0, 1.5, 1.5], [1.5, 1.5, 1.5], [2.0, 1.5, 1.5], [1.0, 2.0, 1.5], [1.5, 2.0, 1.5], [2.0, 2.0, 1.5], [1.0, 1.0, 2.0], [1.5, 1.0, 2.0], [2.0, 1.0, 2.0], [1.0, 1.5, 2.0], [1.5, 1.5, 2.0], [2.0, 1.5, 2.0], [1.0, 2.0, 2.0], [1.5, 2.0, 2.0], [2.0, 2.0, 2.0]])`.

**It seems better to use the load data module in pytorch instead write by ourselves.**

In [51]:
# Data loading and processing
data = scipy.io.loadmat('./data/cylinder_nektar_wake.mat')

U_star = data['U_star'] # N x 2 x T; full data of (u, v) at all x, y, t
P_star = data['p_star'] # N x T; full data of p at all x, y, t
t_star = data['t'] # T x 1; t corrdinates of the full data
XY_star = data['X_star'] # N x 2; (x,y) coordinates of the full data

N = XY_star.shape[0] # N=5000; size of the full data at a fixed t
T = t_star.shape[0] # T=200; size of the full data at a fixed x,y


# Definition of u, v, p
u = torch.tensor(U_star[:,0,:].flatten()[:,None]) # NT x 1
v = torch.tensor(U_star[:,1,:].flatten()[:,None]) # NT x 1
p = torch.tensor(P_star.flatten()[:,None]) # NT x 1

# definition of xyt, u, v, p
xy = torch.tensor(XY_star, requires_grad=True)
t = torch.tensor(t_star, requires_grad=True)

xyxy = torch.cat([torch.tile(xy[:,0:1],(1,T)).flatten().unsqueeze(-1),torch.tile(xy[:,1:2],(1,T)).flatten().unsqueeze(-1)],-1)# xy.size()=(N,2)
tt = torch.tile(t.unsqueeze(0), (N,1)).flatten().unsqueeze(1) # t.size()=(N*T,1)

xyt = torch.cat([xyxy,tt], 1) # NT x 3

In the code block below, we define the training function.

In [52]:
# Training
def train(pinn, optim_method, nIter, lr): 
    start_time = time.time()
#     parameter_list = (pinn.weights + pinn.biases + [param for param in pinn.parameters()]).__iter__()
    parameter_list = pinn.weights + pinn.biases + [param for param in pinn.parameters()]
    if optim_method == "adam":
        optim = torch.optim.Adam(parameter_list,lr)
    elif optim_method == "sgd":
        optim = torch.optim.SGD(parameter_list,lr)
    else:
        print("THIS OPTIMIZATION METHOD IS NOT SUPPORTED.")
        return 0
    for it in range(nIter):
        # Print
        loss_value = pinn.loss_function()
        # loss_value_reg = pinn.loss_function_reg()
        optim.zero_grad()
        # loss_value_reg.backward(retain_graph=True)
        loss_value.backward(retain_graph=True, inputs=parameter_list)
        optim.step()
        lambda_1_value = pinn.lambda_1
        lambda_2_value = pinn.lambda_2
        if it<10 or it % 10 == 0:
            elapsed = time.time() - start_time
            print('It: %d, Loss: %.3e, l1: %.3f, l2: %.5f, Time: %.2f' % 
                  (it, loss_value, lambda_1_value, lambda_2_value, elapsed))
            print(pinn.weights[0].grad.pow(2).sum())
            start_time = time.time()

In code block below, we train the pinn.

In [53]:
# Training Process
N_train = 2000 #5000
N_test = 1000
    
layers = [3, 100, 50, 20, 2]
#[3, 20, 20, 20, 20, 20, 20, 20, 20, 2]
#[3, 300, 200, 100, 2]

nIter = 5 #2000  # original niter is 200000

lr = 0.001

optim_method = "adam"

# Training Data    
idx = np.random.choice(N*T, N_train, replace=False) # Generate a random sample from np.arange(N*T) of size N_train without replacement
xyt_train = xyt[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

# Training
pinn = PhysicsInformedNN_pytorch(xyt_train, u_train, v_train, layers) 
train(pinn, optim_method, nIter, lr)

# Test Data
idx = np.random.choice(N*T, N_test, replace=False) # Generate a random sample from np.arange(N*T) of size N_train without replacement
xyt_test = xyt[idx,:]
u_test = u[idx,:]
v_test = v[idx,:]

# Prediction
u_pred, v_pred, p_pred = pinn(xyt_test)[0:3]
lambda_1_value = pinn.lambda_1
lambda_2_value = pinn.lambda_2

# Error
# error_u = np.linalg.norm(u_test-u_pred,2)/np.linalg.norm(u_test,2)
# error_v = np.linalg.norm(v_test-v_pred,2)/np.linalg.norm(v_test,2)
# error_p = np.linalg.norm(p_test-p_pred,2)/np.linalg.norm(p_test,2)

# error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
# error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100

# print('Error u: %e' % (error_u))    
# print('Error v: %e' % (error_v))    
# print('Error p: %e' % (error_p))    
# print('Error l1: %.5f%%' % (error_lambda_1))                             
# print('Error l2: %.5f%%' % (error_lambda_2))           

It: 0, Loss: 1.826e+03, l1: -0.001, l2: -0.00100, Time: 0.27
tensor(362593.4730, dtype=torch.float64)
It: 1, Loss: 1.778e+03, l1: -0.002, l2: -0.00176, Time: 0.27
tensor(288477.6679, dtype=torch.float64)
It: 2, Loss: 1.743e+03, l1: -0.002, l2: -0.00217, Time: 0.34
tensor(342281.1255, dtype=torch.float64)
It: 3, Loss: 1.708e+03, l1: -0.003, l2: -0.00236, Time: 0.29
tensor(338497.6284, dtype=torch.float64)
It: 4, Loss: 1.670e+03, l1: -0.003, l2: -0.00241, Time: 0.28
tensor(278975.2193, dtype=torch.float64)
